In [2]:
#pip install pyathena

In [3]:
#Libraries
from pyathena import connect
import pandas as pd
import datetime as dt
from datetime import datetime

In [4]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [5]:
#Connection to daily_check (based on receiver)
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')
df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

/tmp/ipykernel_136/2235117015.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)


In [6]:
df['amount'].sum()

25110624454.4149

In [7]:
#Connection to daily_forex 
forex_table = 'last_daily_forex'
df_rates = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{forex_table};', conn)

/tmp/ipykernel_136/2405263410.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rates = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{forex_table};', conn)


In [8]:
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Grouping by 'payer' and 'country' concatenated for this level of granularity
df['payer_country'] = df['payer'] + '_' + df['country']

In [9]:
df = df[df['date'] != '2020-12-31'] # Excluyo el 31-12-2020

In [10]:
def aging_filter(df):
    # Find the last date in the sample
    last_date_sample = df['date'].max()

    # Calculate the limit date, one day before the last date in the sample
    limit_date = last_date_sample - pd.Timedelta(days=1)

    # Find the first and last date per 'payer_country' and calculate total_amount and count
    result = (
        df.groupby('payer_country')
        .agg(
            first_date=('date', 'min'),
            last_date=('date', 'max'),
            total_amount=('amount', 'sum'),
            total_transactions=('tx', 'sum')  # Add this line for total_transactions
        )
        .reset_index()
    )

    # Calculate the 'age_payer' variable accurately using the calculated limit_date
    result['age_payer'] = ((limit_date - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'active_payer' variable
    result['active_time'] = ((result['last_date'] - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'inactive' variable
    result['inactive_time'] = ((limit_date - result['last_date']).dt.days / 30).round(2)

    # Sort the final DataFrame by 'total_amount' from highest to lowest
    result = result.sort_values(by='total_amount', ascending=False)

    # Filter the DataFrame based on conditions and return aging_universe
    aging_universe = result.loc[
        (result.age_payer >= 3) & 
        (result.inactive_time <= 3) & 
        (result.total_amount > 10000) & 
        (result.total_transactions > 50)
    ]

    return aging_universe

In [11]:
# Defining Universe
df = df[df['amount'] != 0] # Excluding 0 (flag A & Flag C), defined in EDA - CHEQUEAR!

df_aging = aging_filter(df) #Filtering 'payer_country' based on Aging notebook
df_filtered = df[df['payer_country'].isin(df_aging['payer_country'])] # Applying aging filters 
df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date

/tmp/ipykernel_136/3736126815.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date


In [12]:
# FOREX - Selecting columns & renaming
df_rates=df_rates.rename(columns={'day': 'date', 'max_feed_price': 'feed_price'})
df_rates=df_rates.loc[:,['date', 'feed_price', 'symbol']]

In [13]:
def get_closing_prices(data, start_date, end_date):
    # Convert the 'FEED_DATE' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    # Filter by date range
    data_filtered = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

    # Sort by 'FEED_DATE' > chronological order
    data_filtered = data_filtered.sort_values(by='date')

    # Group by 'SYMBOL' and 'FEED_DATE', taking the last value of 'FEED_PRICE'
    data_grouped = data_filtered.groupby(['symbol', pd.Grouper(key='date', freq='D')])['feed_price'].last().reset_index()
    
    return data_grouped

In [14]:
start_date = '2021-01-01'
end_date = '2023-10-22'

rates = get_closing_prices(df_rates, start_date, end_date) ### Podriamos omitirla ####

In [15]:
###WE OBTAIN THE LAGS FOR THE EXCHANGE RATE 

def generate_lag_and_variation(df, num_lags):
    # Create columns for each day's lag up to the defined maximum
    for i in range(1, num_lags + 1):
        col_name = f'rate_lag_{i}'
        # Shift the 'feed_price' column grouped by 'symbol'
        df[col_name] = df.groupby('symbol')['feed_price'].shift(i)

    # Calculate the variation columns between consecutive delays
    for i in range(1, num_lags):
        col_name = f'var_rate_lag_{i}'
        # Calculate the difference between consecutive delay columns
        df[col_name] = df[f'rate_lag_{i}'] - df[f'rate_lag_{i + 1}']
    
    return df

In [16]:
rates_number = 14
rates = generate_lag_and_variation(rates, rates_number)
rates['date'] = rates['date'].dt.date #Date format

In [17]:
rates

,symbol,date,feed_price,rate_lag_1,rate_lag_2,rate_lag_3,rate_lag_4,rate_lag_5,rate_lag_6,rate_lag_7,...,var_rate_lag_4,var_rate_lag_5,var_rate_lag_6,var_rate_lag_7,var_rate_lag_8,var_rate_lag_9,var_rate_lag_10,var_rate_lag_11,var_rate_lag_12,var_rate_lag_13
0,USDBRL,2021-01-01,5.1935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USDBRL,2021-01-02,5.1934,5.1935,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USDBRL,2021-01-03,5.2008,5.1934,5.1935,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USDBRL,2021-01-04,5.2976,5.2008,5.1934,5.1935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USDBRL,2021-01-05,5.3244,5.2976,5.2008,5.1934,5.1935,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5120,USDPHP,2023-10-18,56.8872,56.7100,56.8185,56.8485,56.8095,56.8227,56.8275,56.8185,...,-0.0132,-0.0048,0.0090,-0.1157,-0.0108,0.0979,0.1486,-0.0766,0.0307,-0.1480
5121,USDPHP,2023-10-19,56.9022,56.8872,56.7100,56.8185,56.8485,56.8095,56.8227,56.8275,...,0.0390,-0.0132,-0.0048,0.0090,-0.1157,-0.0108,0.0979,0.1486,-0.0766,0.0307
5122,USDPHP,2023-10-20,56.7786,56.9022,56.8872,56.7100,56.8185,56.8485,56.8095,56.8227,...,-0.0300,0.0390,-0.0132,-0.0048,0.0090,-0.1157,-0.0108,0.0979,0.1486,-0.0766
5123,USDPHP,2023-10-21,56.7010,56.7786,56.9022,56.8872,56.7100,56.8185,56.8485,56.8095,...,-0.1085,-0.0300,0.0390,-0.0132,-0.0048,0.0090,-0.1157,-0.0108,0.0979,0.1486


In [18]:
rates_dict = {
    'USDBRL': 'BRAZIL', # Bz Real 
    'USDINR': 'INDIA', # Indian Rupia
    'USDGTQ': 'GUATEMALA', #Quetzal 
    'USDMXN': 'MEXICO', #Mx Peso
    'USDPHP': 'PHILIPPINES' # Ph Peso
}

In [19]:
rates['country'] = rates['symbol'].map(rates_dict)

In [20]:
df1 = pd.merge(df_filtered, rates, on=['date', 'country'], how='left')
df1['date'] = pd.to_datetime(df1['date']).dt.date

In [21]:
### EFFECT OF CANCELED TRANSACTIONS ###

table2_name = 'daily_sales_count_cancelled_v2'##WE LOAD THE BASE WITH CANCELLATIONS
df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2[df2['date'] != '2020-12-31'] # Elimino el registro del 31.12.2020

/tmp/ipykernel_136/597858736.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)


In [22]:
df2['amount'].sum()

25102147362.2899

In [23]:
from sklearn.preprocessing import LabelEncoder

# Grouping by 'payer' and 'country' concatenated for this level of granularity
df2['payer_country'] = df2['payer'] + '_' + df2['country']
# Inicializing LabelEncoder
label_encoder = LabelEncoder()
# Coding ‘PAYER_COUNTRY’ as unique values
df2['payer_country_encoder'] = label_encoder.fit_transform(df2['payer_country'])

In [24]:
# By applying the same aging filter, we can work on the same payer_country universe 
df2 = df2[df2['payer_country'].isin(df_aging['payer_country'])]

In [25]:
def fill_missing_dates(df, start_date, end_date):
    # Create an empty DataFrame with the specified date range
    date_range = pd.date_range(start=start_date, end=end_date)
    df_fill = pd.DataFrame({'date': date_range, 'amount': 0, 'tx_cancelled': 0})
    df_fill['date'] = pd.to_datetime(df_fill['date']).dt.date
    
    # Sort the original DataFrame by 'country', 'payer', and 'date'
    df = df.sort_values(by=['country', 'payer', 'date'])
    
    # Create an empty DataFrame to hold the result
    result_df = pd.DataFrame()

    # Loop through each 'payer_country'
    for payer_country in df['payer_country'].unique():
#        print(payer_country)
        df_aux = df[df['payer_country'] == payer_country]
        # Combining df_aux (payer_country) with df_fill, keeping values from df_aux and filling dates, amount and tx_cancelled 
        merged_df = df_aux.set_index('date').combine_first(df_fill.set_index('date')).reset_index()

        # Fill missing values in 'payer', 'country', 'payer_country', 'payer_country_encoder' columns
        columns_to_fill = ['payer', 'country', 'payer_country', 'payer_country_encoder']
        merged_df[columns_to_fill] = merged_df[columns_to_fill].ffill().bfill()

        result_df = pd.concat([result_df, merged_df], ignore_index=True)

    
    return result_df

In [26]:
#start_date and end_date variables defined previously
start_date = '2021-01-01'
end_date = '2023-10-21'

# Call the function with the specified start_date and end_date
df_full = fill_missing_dates(df2, start_date, end_date)

In [27]:
def generate_tx_lags_and_variation(df, tx_count):
    """
    Generate lag columns for cancelled transactions and their variations.

    Args:
    - df: DataFrame containing transaction data
    - tx_count: Number of periods for lag calculation

    Returns:
    - df: DataFrame with added lag and variation columns
    """
    # Sort the dataset based on country, payer, and date
    df = df.sort_values(by=['country', 'payer', 'date'])

    # Create columns for each day's lag up to the defined maximum
    for i in range(1, tx_count + 1):
        col_name = f'tx_cancelled_lag_{i}'
        # Shift the 'tx_cancelled' column grouped by 'country' and 'payer'
        df[col_name] = df.groupby(['country', 'payer'])['tx_cancelled'].shift(i)

    # Calculate the variation columns between consecutive delays
    for i in range(1, tx_count):
        col_name = f'var_tx_cancelled_lag_{i}'
        # Calculate the difference between consecutive lag columns
        df[col_name] = df[f'tx_cancelled_lag_{i}'] - df[f'tx_cancelled_lag_{i + 1}']

    return df


In [28]:
# Call the function and assign the result back to df2
tx_cancelled_lags = 7
df2 = generate_tx_lags_and_variation(df_full, tx_cancelled_lags)

In [29]:
df2['date'] = df2['date'].dt.date

In [30]:
df2['amount'].sum()

24303285928.466

In [31]:
df_final = pd.merge(df1,df2, on=['date','payer','country', 'amount'], how='outer')
df_final['date'] = pd.to_datetime(df_final['date'])

In [32]:
###WE SHOULD KEEP THE VARIABLES THAT HAVE IMPUTED MISSINGS IN AMOUNT AND TX_CANCELLED (DAYS WITHOUT OPERATIONS) BUT KEEP THE MISSINGS IN RATES
df_final=df_final.loc[:,['date', 'payer', 'country', 'amount', 
                         'var_rate_lag_1', 'var_rate_lag_2', 'var_rate_lag_3',
                         'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6', 'var_rate_lag_7',
                         'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10',
                         'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13',
                         'payer_country_encoder','payer_country_y',
                         'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
                         'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6']]
df_final = df_final.rename(columns={'payer_country_y': 'payer_country'})

In [33]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133120 entries, 0 to 133119
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    133120 non-null  datetime64[ns]
 1   payer                   133120 non-null  object        
 2   country                 133120 non-null  object        
 3   amount                  133120 non-null  float64       
 4   var_rate_lag_1          33730 non-null   float64       
 5   var_rate_lag_2          33701 non-null   float64       
 6   var_rate_lag_3          33673 non-null   float64       
 7   var_rate_lag_4          33644 non-null   float64       
 8   var_rate_lag_5          33617 non-null   float64       
 9   var_rate_lag_6          33589 non-null   float64       
 10  var_rate_lag_7          33561 non-null   float64       
 11  var_rate_lag_8          33534 non-null   float64       
 12  var_rate_lag_9          33505 

In [34]:
### DUMMIES ####

# Weekend indicator (dummy)
df_final['weekend'] = (df_final['date'].dt.dayofweek >= 4).astype(int)

In [35]:
#WE ADD ONE MORE DAY AFTER CHRISTMAS, NEW YEAR'S EVE AND US INDEPENDENCE (BETTER CAPTURES THE DISTURBANCE).
# Special dates list - Format: Month-day 
special_dates_list = [ '07-04', #US independence day 
                      '12-25', #Christmas
                      '05-11' # Mexico & Colombia - Mother's day
                     '01-01', #New Year
                     '01-02',
                     '12-26',
                     '07-05'] 

# Applying format process special_dates function
special_dates_list_processed = [(int(date.split('-')[0]), int(date.split('-')[1])) for date in special_dates_list]

In [36]:
# To check special dates defined by list
def mark_special_dates(row):
    if (row['date'].month, row['date'].day) in special_dates_list_processed:
        return 1
    else:
        return 0

In [37]:
# Marking special dates in df
df_final['special_dates'] = df_final.apply(lambda row: mark_special_dates(row), axis=1)

In [38]:
df_final['amount'].sum()

24303285928.466

In [39]:
df_final['amount'].sum()

24303285928.466

In [40]:
# Setting DS timeframe

df_14lags = df_final.loc[df_final.date<'2023-07-01']

In [41]:
df_14lags.columns

Index(['date', 'payer', 'country', 'amount', 'var_rate_lag_1',
       'var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5',
       'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9',
       'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12',
       'var_rate_lag_13', 'payer_country_encoder', 'payer_country',
       'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2',
       'var_tx_cancelled_lag_3', 'var_tx_cancelled_lag_4',
       'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 'weekend',
       'special_dates'],
      dtype='object')

In [52]:
df_14lags['date'].max()

Timestamp('2023-06-30 00:00:00')

In [44]:
# TARGET CSV
target_ts_amount=df_14lags.loc[:, ['payer_country_encoder', 'date', 'amount']]
target_ts_amount.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp', 'amount': 'target_value'}, inplace=True)

In [45]:
# RELATED TS
###GENERATE THE DATASET OF RELATED TIME SERIES (UP TO 23 VARIABLES+ID_ITEM+TIMESTAMP-FIRST THESE TWO VARIABLES)
related_ts_columns = ['payer_country_encoder', 'date', 
                      'var_rate_lag_1', 'var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5',
       'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12',
       'var_rate_lag_13', 'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2',
       'var_tx_cancelled_lag_3', 'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 'weekend', 'special_dates']

related_ts_ds = df_14lags[related_ts_columns]
related_ts_ds.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp'}, inplace=True)

/tmp/ipykernel_136/3502354415.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  related_ts_ds.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp'}, inplace=True)


In [ ]:
related_ts_ds.columns

In [ ]:
 [item_id, timestamp, var_rate_lag_1, var_rate_lag_2, var_rate_lag_3, var_rate_lag_4, var_rate_lag_5, var_rate_lag_6, var_rate_lag_7, var_rate_lag_8, var_rate_lag_9, var_rate_lag_10, var_rate_lag_11, var_rate_lag_12, var_rate_lag_13, var_tx_cancelled_lag_1, var_tx_cancelled_lag_2, var_tx_cancelled_lag_3, var_tx_cancelled_lag_4, var_tx_cancelled_lag_5, var_tx_cancelled_lag_6, weekend, special_dates]

In [46]:
###GENERATE A DATASET WITH METADATA(COUNTRY AND PAYER)
metadata_ts=df_14lags.loc[:, ['payer_country_encoder', 'country', 'payer']]
metadata_ts.rename(columns={'payer_country_encoder': 'item_id'}, inplace=True)

In [48]:
#pip install awswrangler

In [49]:
#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#
##ROUTINES FOR EXPORTING CSV TO VIAMERICAS S3##
import awswrangler as wr
file_name_target = 'target_14lags_Junio23.csv'

wr.s3.to_csv(
    df=target_ts_amount,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/target/{file_name_target}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/target/target_14lags_Junio23.csv'],
 'partitions_values': {}}

In [50]:
file_name_ts = 'related_ts_14lags_Junio23.csv'

wr.s3.to_csv(
    df=related_ts_ds,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/{file_name_ts}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/related_ts_14lags_Junio23.csv'],
 'partitions_values': {}}

In [51]:
file_name_metadata = 'metadata_14lags_Junio23.csv'

wr.s3.to_csv(
    df=metadata_ts,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/{file_name_metadata}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/metadata_14lags_Junio23.csv'],
 'partitions_values': {}}